In [ ]:
import pickle
def load_data(filename):
    data = pickle.load(open(filename, 'rb'))
    x_text = []
    labels = []
    for i in range(len(data)):
        if(HASH_REMOVE):
            x_text.append(p.tokenize((data[i]['text']).encode('utf-8')))
        else:
            x_text.append(data[i]['text'])
        labels.append(data[i]['label'])
    return x_text,labels

def get_filename(dataset):
    global N_CLASS, HASH_REMOVE
    if(dataset=="twitter"):
        filename = "../input/formspring1/twitter_data.pkl"
        N_CLASS = 3
        HASH_REMOVE = False
    elif(dataset=="formspring"):
        N_CLASS = 2
        filename = "../input/formspring_data.pkl"
        HASH_REMOVE = False
    elif(dataset=="wiki"):
        N_CLASS = 2
        filename = "../input/formspring1/wiki_data.pkl"
        HASH_REMOVE = False
    return filename


In [ ]:
data = "formspring"
x_text, labels = load_data(get_filename(data)) 

In [ ]:
import re


In [ ]:
x_corp = []
for x in range(len(x_text)):
    words = re.split(r'\W+', x_text[x])
    words = [word.lower() for word in words]
    x_corp.append(words)
    
x_corp = [[' '.join(i)] for i in x_corp]

In [ ]:
import pandas as pd

for x in range(len(labels)):
    if labels[x] ==0 :
        labels[x] = 1
    else:
        labels[x] = 2


data = pd.concat([pd.DataFrame(x_corp),pd.DataFrame(labels)],axis=1)

data.columns = ['text', 'label']

In [ ]:
count=0
final = []
for x in range(len(x_corp)):
    if data.iloc[x,0] == ''or data.iloc[x,0] == ' ' or data.iloc[x,1] == '' or data.iloc[x,1] == ' ':
        count+=1
        continue
    final.append(data.iloc[x,:].values)
       
print(count)

In [ ]:
!pip install tiny-tokenizer
!pip install flair

In [ ]:
embedding = DocumentPoolEmbeddings([WordEmbeddings('glove'),ELMoEmbeddings()], fine_tune_mode='nonlinear')

In [ ]:
import os
from flair.data import Sentence
for i in  range(len(final)):
    sentence = final[i][0]
    sentence = Sentence(sentence)
    embedding.embed(sentence)
    final[i][0] = sentence.embedding.tolist()
    os.system( 'cls' )
    print(i)
    os.system( 'cls' )
   

In [ ]:
import numpy as np
from keras import models
from keras.layers import Dense,Conv1D,Dropout,Flatten,Dense,MaxPooling1D
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
np.random.seed(0)

def create_network():
    
    model = models.Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(3172,1)))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    
    return model

In [ ]:
neural_network = KerasClassifier(build_fn=create_network, epochs=200, batch_size=100, verbose=1)

In [ ]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
results = cross_val_score(neural_network, X, y, cv=10)
print(results.mean())

In [ ]:
results

In [ ]:
X = np.vstack(final[:,0])
y = np.vstack(final[:,1])

In [ ]:
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

In [ ]:
X = X.reshape(12759,3172,1)

In [ ]:
final.to_csv('embeddings.csv',index=None)

In [ ]:
import numpy as np
train, validate, test = np.split(final.sample(frac=1), [int(.7*len(final)), int(.85*len(final))])

In [ ]:
train.to_csv('train.csv',index=None)
validate.to_csv('dev.csv',index=None)
test.to_csv('test.csv',index=None)

In [ ]:
!pip install tiny-tokenizer
!pip install flair

In [ ]:
import pandas as pd
dataset = pd.read_csv('/kaggle/working/train.csv')

In [ ]:
count=0
final = []
for x in range(len(dataset)):
    if dataset.iloc[x,0] != ''or dataset.iloc[x,0] != ' ' or dataset.iloc[x,1] != ''or dataset.iloc[x,1] != ' ':
        final.append(dataset.iloc[x,:].values)

In [ ]:
len(dataset)

In [ ]:
from flair.data import Corpus
data_folder ='/kaggle/working'
column_name_map = {0: "text",1: "label_topic"}

In [ ]:
from flair.data import Sentence,FlairDataset
from flair.data import Token, segtok_tokenizer
from pathlib import Path
from typing import List, Dict, Union, Callable
import logging
import csv

class CSVClassificationData(FlairDataset):
    def __init__(
        self,
        path_to_file: Union[str, Path],
        column_name_map: Dict[int, str],
        max_tokens_per_doc: int = -1,
        max_chars_per_doc: int = -1,
        tokenizer=segtok_tokenizer,
        in_memory: bool = True,
        skip_header: bool = False,
        **fmtparams,
    ):
    

        if type(path_to_file) == str:
            path_to_file: Path = Path(path_to_file)

        assert path_to_file.exists()

        # variables
        self.path_to_file = path_to_file
        self.in_memory = in_memory
        self.tokenizer = tokenizer
        self.column_name_map = column_name_map
        self.max_tokens_per_doc = max_tokens_per_doc
        self.max_chars_per_doc = max_chars_per_doc

        # different handling of in_memory data than streaming data
        if self.in_memory:
            self.sentences = []
        else:
            self.raw_data = []

        self.total_sentence_count: int = 0

        # most data sets have the token text in the first column, if not, pass 'text' as column
        self.text_columns: List[int] = []
        for column in column_name_map:
            if column_name_map[column] == "text":
                self.text_columns.append(column)

        with open(self.path_to_file) as csv_file:

            csv_reader = csv.reader(csv_file, **fmtparams)

            if skip_header:
                next(csv_reader, None)  # skip the headers

            for row in csv_reader:
                # test if format is OK
                wrong_format = False
                for text_column in self.text_columns:
                    if text_column >= len(row):
                        wrong_format = True

                if wrong_format:
                    continue
                

                # test if at least one label given
                has_label = False
                for column in self.column_name_map:
                    if self.column_name_map[column].startswith("label") and row[column]:
                        has_label = True
                        break
                

                if not has_label:
                    continue

                if self.in_memory:
                    
                    print('yes')

                    text = " ".join(
                        [row[text_column] for text_column in self.text_columns]
                    )
                    
                    if self.max_chars_per_doc > 0:
                        text = text[: self.max_chars_per_doc]

                    sentence = Sentence(text, use_tokenizer=self.tokenizer)

                    for column in self.column_name_map:
                        if (
                            self.column_name_map[column].startswith("label")
                            and row[column]
                        ):
                            sentence.add_label(row[column])

                    if 0 < self.max_tokens_per_doc < len(sentence):
                        sentence.tokens = sentence.tokens[: self.max_tokens_per_doc]
                    self.sentences.append(sentence)

                else:
                    print('no')
                    self.raw_data.append(row)

                self.total_sentence_count += 1
                print(self.total_sentence_count)

    def is_in_memory(self) -> bool:
        return self.in_memory

    def __len__(self):
        return self.total_sentence_count

    def __getitem__(self, index: int = 0) -> Sentence:
        if self.in_memory:
            return self.sentences[index]
        else:
            row = self.raw_data[index]

            text = " ".join([row[text_column] for text_column in self.text_columns])

            if self.max_chars_per_doc > 0:
                text = text[: self.max_chars_per_doc]

            sentence = Sentence(text, use_tokenizer=self.tokenizer)
            for column in self.column_name_map:
                if self.column_name_map[column].startswith("label") and row[column]:
                    sentence.add_label(row[column])

            if 0 < self.max_tokens_per_doc < len(sentence):
                sentence.tokens = sentence.tokens[: self.max_tokens_per_doc]

            return sentence

log = logging.getLogger("flair")

class CSVClassificationCorp(Corpus):
    def __init__(
        self,
        data_folder: Union[str, Path],
        column_name_map: Dict[int, str],
        train_file=None,
        test_file=None,
        dev_file=None,
        tokenizer: Callable[[str], List[Token]] = segtok_tokenizer,
        max_tokens_per_doc=-1,
        max_chars_per_doc=-1,
        in_memory: bool = False,
        skip_header: bool = False,
        **fmtparams,
    ):
    

        if type(data_folder) == str:
            data_folder: Path = Path(data_folder)

        if train_file is not None:
            train_file = data_folder / train_file
        if test_file is not None:
            test_file = data_folder / test_file
        if dev_file is not None:
            dev_file = data_folder / dev_file

        # automatically identify train / test / dev files
        if train_file is None:
            for file in data_folder.iterdir():
                file_name = file.name
                if "train" in file_name:
                    train_file = file
                if "test" in file_name:
                    test_file = file
                if "dev" in file_name:
                    dev_file = file
                if "testa" in file_name:
                    dev_file = file
                if "testb" in file_name:
                    test_file = file

        log.info("Reading data from {}".format(data_folder))
        log.info("Train: {}".format(train_file))
        log.info("Dev: {}".format(dev_file))
        log.info("Test: {}".format(test_file))

        train: Dataset = CSVClassificationData(
            train_file,
            column_name_map,
            tokenizer=tokenizer,
            max_tokens_per_doc=max_tokens_per_doc,
            max_chars_per_doc=max_chars_per_doc,
            in_memory=in_memory,
            skip_header=skip_header,
            **fmtparams,
        )

        if test_file is not None:
            test: Dataset = CSVClassificationData(
                test_file,
                column_name_map,
                tokenizer=tokenizer,
                max_tokens_per_doc=max_tokens_per_doc,
                max_chars_per_doc=max_chars_per_doc,
                in_memory=in_memory,
                skip_header=skip_header,
                **fmtparams,
            )
        else:
            train_length = len(train)
            test_size: int = round(train_length / 10)
            splits = random_split(train, [train_length - test_size, test_size])
            train = splits[0]
            test = splits[1]

        if dev_file is not None:
            dev: Dataset = CSVClassificationData(
                dev_file,
                column_name_map,
                tokenizer=tokenizer,
                max_tokens_per_doc=max_tokens_per_doc,
                max_chars_per_doc=max_chars_per_doc,
                in_memory=in_memory,
                skip_header=skip_header,
                **fmtparams,
            )
        else:
            train_length = len(train)
            dev_size: int = round(train_length / 10)
            splits = random_split(train, [train_length - dev_size, dev_size])
            train = splits[0]
            dev = splits[1]

        super(CSVClassificationCorp, self).__init__(
            train, dev, test, name=data_folder.name
        )



In [ ]:
corpus: Corpus = CSVClassificationCorp(data_folder,
                                         column_name_map,
                                         skip_header=True,
                                         delimiter=',', 
) 

In [ ]:
print(corpus)

In [ ]:
label_dict = corpus.make_label_dictionary()

In [ ]:
from flair.embeddings import WordEmbeddings, ELMoEmbeddings, FlairEmbeddings,DocumentRNNEmbeddings,DocumentPoolEmbeddings



In [ ]:
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
document_embeddings: document_embeddings = DocumentPoolEmbeddings([WordEmbeddings('glove'),ELMoEmbeddings()], fine_tune_mode='nonlinear')

In [ ]:
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

trainer = ModelTrainer(classifier, corpus)

trainer.train('resources/taggers/ag_news',
              learning_rate=0.05,
              mini_batch_size=25,
              anneal_factor=0.05,
              patience=3,
              max_epochs=100)

In [ ]:
sentence = Sentence('you are a whore')

# predict class and print
classifier.predict(sentence)

print(sentence.labels)

In [ ]:
import os
os.chdir(r'/kaggle/working')

In [ ]:
from IPython.display import FileLink
FileLink(r'train.pickle')

In [ ]:
import pandas as pd
dataset = pd.read_csv('embeddings.csv')

In [ ]:

with open('train.pickle', 'wb') as f:
    pickle.dump([X,y], f)